In [1]:
import numpy as np
import pandas as pd
import os
import librosa

In [4]:
aud_dir = '../audioData/BVCVoice/'
meta_df = pd.read_excel(os.path.join(aud_dir, 'BVC_Voice_Bio_Public.xlsx'))

In [17]:
audio_files = []
for dir, subdir, files in (os.walk(aud_dir)):
    if len(files) > 0:
        for file in files:
            if file.split('.')[1] == 'wav':
                audio_files.append(os.path.join(dir, file))

In [32]:
meta_df.columns

Index(['New_ID', 'Sex', 'Age', 'Ethnicity', 'Unnamed: 4', 'Unnamed: 5'], dtype='object')

In [34]:
gender_list = []
age_list = []

for file in audio_files:
    speaker_id = (file.split('/')[-1].split('_')[2])
    gender_list.append(meta_df[meta_df['New_ID'] == int(speaker_id)]['Sex'].values[0])
    age_list.append(meta_df[meta_df['New_ID'] == int(speaker_id)]['Age'].values[0])

In [37]:
bvc_df = pd.DataFrame({'audio_file': audio_files, 'gender': gender_list, 'age': age_list})

In [77]:
import model_utils as mu

gender_models = mu.MLModelManager()
gender_models.load_classification_MLModels('trained_models/speakerGender_KaggleVAD/')


No saved model found for Logistic Regression at trained_models/speakerGender_KaggleVAD/Logistic Regression.joblib
No saved model found for Decision Tree at trained_models/speakerGender_KaggleVAD/Decision Tree.joblib
No saved model found for Random Forest at trained_models/speakerGender_KaggleVAD/Random Forest.joblib
No saved model found for SVM at trained_models/speakerGender_KaggleVAD/SVM.joblib
No saved model found for KNN at trained_models/speakerGender_KaggleVAD/KNN.joblib
No saved model found for AdaBoost at trained_models/speakerGender_KaggleVAD/AdaBoost.joblib
No saved model found for Gradient Boosting at trained_models/speakerGender_KaggleVAD/Gradient Boosting.joblib


In [42]:
# Function to extract features (same as before)
def extract_features(file_path):
    y, sr = librosa.load(file_path)
    mfccs = np.mean(librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13).T, axis=0)
    chroma = np.mean(librosa.feature.chroma_stft(y=y, sr=sr, n_fft=512).T, axis=0)
    mel = np.mean(librosa.feature.melspectrogram(y=y, sr=sr, n_fft=512).T, axis=0)
    contrast = np.mean(librosa.feature.spectral_contrast(y=y, sr=sr, n_fft=512).T, axis=0)
    tonnetz = np.mean(librosa.feature.tonnetz(y=y, sr=sr).T, axis=0)
    return np.hstack([mfccs, chroma, mel, contrast, tonnetz])

In [43]:
scaler = mu.scaler
label_encoder = mu.label_encoder

In [45]:
features = np.array([extract_features(file_path) for file_path in bvc_df['audio_file']])
labels = bvc_df['gender']

scaled_features = scaler.transform(features)
#encoded_labels = label_encoder.transform(labels)

/work/dpandya/miniconda3/envs/whisperx_/lib/python3.10/site-packages/librosa/core/spectrum.py:266: UserWarning: n_fft=1024 is too large for input signal of length=689
  warnings.warn(
/work/dpandya/miniconda3/envs/whisperx_/lib/python3.10/site-packages/librosa/core/spectrum.py:266: UserWarning: n_fft=1024 is too large for input signal of length=862
  warnings.warn(
/work/dpandya/miniconda3/envs/whisperx_/lib/python3.10/site-packages/librosa/core/spectrum.py:266: UserWarning: n_fft=1024 is too large for input signal of length=431
  warnings.warn(
/work/dpandya/miniconda3/envs/whisperx_/lib/python3.10/site-packages/librosa/core/spectrum.py:266: UserWarning: n_fft=1024 is too large for input signal of length=957
  warnings.warn(
/work/dpandya/miniconda3/envs/whisperx_/lib/python3.10/site-packages/librosa/core/spectrum.py:266: UserWarning: n_fft=1024 is too large for input signal of length=479
  warnings.warn(
/work/dpandya/miniconda3/envs/whisperx_/lib/python3.10/site-packages/librosa/cor

ValueError: y contains previously unseen labels: "'Male'"

In [73]:
#label_encoder.transform([(labels.iloc[1345].strip().strip('"').strip("'"))])
for i in range(len(labels)):
    labels[i] = labels[i].strip().strip('"').strip("'")

encoded_labels = label_encoder.transform(labels)

/tmp/ipykernel_4012185/969994819.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  labels[i] = labels[i].strip().strip('"').strip("'")
/tmp/ipykernel_4012185/969994819.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  labels[i] = labels[i].strip().strip('"').strip("'")
/tmp/ipykernel_4012185/969994819.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  labels[i] = labels[i].strip().strip('"').strip("'")
/tmp/ipykernel_4012185/96999

In [80]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

eval_results = {}
for name, model in gender_models.models.items():
    model.fit(scaled_features, encoded_labels)
    y_pred = model.predict(scaled_features)

    eval_results[name] = {
        'Accuracy': accuracy_score(encoded_labels, y_pred),
        'Precision': precision_score(encoded_labels, y_pred),
        'Recall': recall_score(encoded_labels, y_pred),
        'F1-Score': f1_score(encoded_labels, y_pred)
    }
    print(f"---xxx---{name} is evaluated---xxx---")

In [79]:
#pd.DataFrame(eval_results).to_csv('trained_models/speakerGender_KaggleAndVoxCeleb/report_bvcvoice.csv', index=False)
pd.DataFrame(eval_results)

""


In [82]:
bvc_df.to_csv('bvcVoice_genderAge.csv', index=False)